In [2]:
import torch
import torchxrayvision as xrv
import torchvision
import numpy as np
import skimage.io
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Φορτώνεις DenseNet-121 με weights εκπαιδευμένα σε CheXpert
model = xrv.models.DenseNet(weights="densenet121-res224-chex").to(device)
model.eval()

print("Pathologies:", model.pathologies)  # λίστα με labels CheXpert-style


C:\Users\chtzo\.conda\envs\Hackathon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/chex-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O C:\Users\chtzo\.torchxrayvision\models_data/chex-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]
Pathologies: ['Atelectasis', 'Consolidation', '', 'Pneumothorax', 'Edema', '', '', 'Effusion', 'Pneumonia', '', 'Cardiomegaly', '', '', '', 'Lung Lesion', 'Fracture', 'Lung Opacity', 'Enlarged Cardiomediastinum']


In [3]:
import torch.nn.functional as F

# transforms της βιβλιοθήκης για X-ray
transform = torchvision.transforms.Compose([
    xrv.datasets.XRayCenterCrop(),
    xrv.datasets.XRayResizer(224)
])

def predict_image_pil(pil_img: Image.Image):
    # σε grayscale (μονό channel)
    img = np.array(pil_img.convert("L"))

    # normalize όπως στα docs
    img = xrv.datasets.normalize(img, 255)   # 8-bit -> [-1024, 1024]
    img = img[None, ...]                     # [1, H, W] για single channel

    img = transform(img)                     # crop + resize (numpy)
    img = torch.from_numpy(img).unsqueeze(0).to(device)  # [1, 1, 224, 224]

    with torch.no_grad():
        logits = model(img)                  # [1, num_pathologies]
        probs = torch.sigmoid(logits)[0].cpu().numpy()   # σε [0,1]

    return dict(zip(model.pathologies, probs.tolist()))
